In [ ]:
# set this to the path where this notebook is located
package_dir = "/atlas/scratch0/pagessin/dev/annulus_21.9/athena/Tracking/TrkDetDescr/AnnulusDebug"
# set this to the path where you ran the AnnulusTest algorithm from
run_dir = "/atlas/scratch0/pagessin/dev/annulus_21.9/run_2018-12-07"
# enable / disable printing of c++ code, code comments
do_print_cpp = False

In [ ]:
from IPython.display import Image
import os
def img(filename, *args, **kwargs): 
    return Image(filename=os.path.join(package_dir, "doc", filename), *args, **kwargs)

# ITk Strip Endcap module geometry

The upgraded InnerTracker will feature a newly designed strip-based silicon tracking geometry. For the barrel, the overall module shape stays the same, for the endcaps, the shape of the module departs from the symmetric design, which is rotated by a stereo angle to allow 2D measurements. 
The new strip modules have a built-in stereo angle. This manifests itself in an irregular shape, where the radial and the angular bounds **do not share the same origin**.

<sup>[1](#petalscds)</sup>

In [ ]:
img("petal1.png", width=300)


The current implementation of this shape uses a rotated cartesian, as seen in the image above. This allows trivially defining the radial bounds by simple circles, but requires the angular sides to be parametrized lines with an offset from the origin of the coordinate system. 

The approach causes difficulties when considering measurement covariances in the local frame of the module. The underlying measurement uncertainty is caused by the physical size of the semiconductor segmentation, the strips. It is given be the length of the the strips, and their pitch in $\phi$. In the cartesian implementation, the strip pitch is evaluated in the center, as if the strips were straight, and then scaled up and rotated, to account for larger or smaller (cartesian) strip pitch and effective stereo angle along the $y$-axis.

Using a polar coordinate system as the local frame, this problem goes away. The covariance defined by the semiconductor geometry is trivially

\begin{equation}
C = \frac{1}{12}\left[\begin{array}{cc}
p_{r}^2 & 0 \\
0 & p_\phi^2
\end{array}\right]
\end{equation}

with the strip pitches $p_r$ and $p_\phi$. During a fit, track parameters can then simply be transformed into the local strip polar coordinate frame, and the residual and $\chi^2$ calculation becomes trivial. In principle, this could even become a pure 1D measurment, dropping the $r$ component, and using only the $\phi$ angle, and corresponding pitch induced covariance.

This requires the local frame to be aligned with strip coordinate system. The radial bounds, however, are not defined with respect to this system. To remedy this, the cartesian offset between the two coordinate system origins is identified:

In [ ]:
img("annulus.png", width=300)

The coordinate system around which the concentric radial inner and outer bounds are defined is subsequently called the **MODULE SYSTEM**, while the point where the angular straight-line boundaries of the shape intersect is the origin of the **STRIP SYSTEM**.

This allows conversion between coordinate systems by shifting the origin in the cartesian frame. A method to directly convert the $r$ coordinate of a point from the strip system to the module system, without explicitly converting to cartesian coordinates in-between is described below.


<a name="petalscds">1</a>: Building a Stereo-angle into strip-sensors for the ATLAS-Upgrade Inner-Tracker Endcaps

Some necessary imports for use below:

In [ ]:
%matplotlib inline
import shapely
from matplotlib import pyplot as plt
import matplotlib as mpl
from shapely.geometry import LinearRing, LineString, Polygon, Point, MultiLineString, MultiPoint
from shapely.ops import linemerge, polygonize, transform
from shapely import affinity
from IPython.display import display, Markdown, Latex, Math, display_markdown
from descartes import PolygonPatch
import numpy as np
from datetime import datetime
from IPython.core.pylabtools import figsize
from math import pi, sin, cos, atan2, sqrt, acos, asin, ceil
import pandas as pd
import sympy as sp
from sympy.geometry import Ray, Circle, intersection
import sympy.matrices
from tqdm import tqdm_notebook as tqdm
sp.init_printing()

# Helper functions and classes

This defines some helpers that are used for linear algebra manipulation below, and pretty printing of latex and c++ code.

In [ ]:
%run -i $package_dir/doc/notebook_helpers.py

def print_cpp(*args):
    if do_print_cpp:
        s = " ".join(map(str, args))
        display(Markdown("```cpp\n%s\n```"%s))
        
def jacobian(fs, vs):
    def gen(i, j):
        return sp.Derivative(fs[i], vs[j])
    m = sp.Matrix(len(fs), len(vs), gen)
    return m

def save(fig, filename):
    fig.tight_layout()
    fig.savefig(os.path.join(package_dir, "doc", filename))

# Jacobians for covariance transformation

For covariance based inside checks of the bounds, converting covariances matrices into the two coordinate systems **STRIP** and **MODULE** is required. The input is assumed to be given in the **STRIP** coordinate system.
This requires a Jacobian, which can be applied to the covariance matrix to transform it. The covariance $\hat\sigma$ transforms as 

$$\hat\sigma' = J \hat\sigma J^T$$

$J$ is calculated by building an expression converting coordinates from **STRIP-PC** to **STRIP-XY** to **MODULE-XY** and finally to **MODULE-PC**, and then calculating the various derivatives. The Jacobian has to be calculated at a given point in order to be correct.
There is an additional optional rotation $\Delta\phi$.

Set up symbols $x$, $y$ as local cartesian coordinates. The circle in polar coordinates is then:

In [ ]:
x, y, vx, vy = sp.symbols('x y v_x, v_y')
vx, vy = sp.sqrt(x**2 + y**2), 2*sp.atan( y / (sp.sqrt(x**2 + y**2) + x) )
vec = sp.Matrix([vx, vy])

latex(r"$\vec v = %s$", vec)
print_cpp("// " & ("v = " + LineArray(pretty(sp.Matrix(sp.symbols("r' phi'")))) + " = " + LineArray(pretty(vec))))

The Jacobian $J$ is generically:

In [ ]:
fx = sp.Function("f_x")(x, y)
fy = sp.Function("f_y")(x, y)


Jac_gen = jacobian(sp.symbols("f_x f_y"), sp.symbols("x y"))
latex(r"$J_\text{gen} = %s$", Jac_gen)
print_cpp("// " & "Jgen = " + LineArray(pretty(Jac_gen)))


This gives without any further substitutions:

In [ ]:
J = jacobian([vx, vy], [x, y])
latex(r"$$J = %s$$", J)

However, we want to go from **STRIP** to **MODULE** system, which means we need to describe $x, y$ in *shifted* polar coordinates. 

$$
\left(\begin{matrix}
x \\ 
y 
\end{matrix}\right)
= \left(\begin{matrix}
r \cos(\phi - \Delta\phi) \\
r \sin(\phi - \Delta\phi)
\end{matrix}\right)
$$

where $r$, $\phi$ are the polar coordinates of the point and $\Delta\phi$ is a rotation that might be applied.
$x, y$ are still in the **STRIP** system at this point. We transform into **MODULE** by shifting the origin:

$$
\left(
\begin{matrix}
x_2 \\ y_2
\end{matrix}
\right) = 
\left(
\begin{matrix}
x \\ y
\end{matrix}
\right) + \vec O
=
\left(
\begin{matrix}
r \cos(\phi - \Delta\phi) + O_x \\
r \sin(\phi - \Delta\phi + O_y
\end{matrix}
\right)
$$

This vector needs to be calculated back to polar coordinates.

In [ ]:
# produce comment for equation above (x, y)
x, y, r, phi, dPhi, O_x, O_y = sp.symbols("x y r phi dPhi O_x O_y")
a = sp.Matrix([x, y])
b = sp.Matrix([r*sp.cos(phi - dPhi) + O_x, r*sp.sin(phi - dPhi) + O_y])
print_cpp("// " & (LineArray(pretty(a)) + " = " + LineArray(pretty(b))))

In [ ]:
# set up symbols
r_strip, phi_strip, dphi = sp.symbols("r_{strip} \phi_{strip} \Delta\phi")
r_mod, phi_mod = sp.symbols("r_{mod} \phi_{mod}")
Ox, Oy = sp.symbols("O_x O_y")

# calculate polar coordinates for 
x2, y2 = r_strip * sp.cos(phi_strip - dphi) + Ox, r_strip * sp.sin(phi_strip - dphi) + Oy
ml = sp.Matrix([r_mod, phi_mod])
mr = sp.Matrix([vx.subs({x: x2, y: y2}), vy.subs({x: x2, y: y2})])
latex(r"""in **MODULE** polar coordinate system:

$%s = %s$""", ml, mr)

ml, mr = [m.subs({r_mod: "rMod", phi_mod: "phiMod", r_strip: "rStrip", phi_strip: "phiStrip", dphi: "dPhi"}) for m in (ml, mr)]

print_cpp("// " & ( LineArray(pretty(ml)) + " = " + LineArray(pretty(mr)) ))

In [ ]:
J = jacobian([r_mod, phi_mod], [r_strip, phi_strip])
latex(r"$J = %s$", J)
print_cpp("// " & "J = " + LineArray(pretty(J.subs({r_mod: "rMod", phi_strip: "phiStrip", phi_mod: "phiMod"}))))


latex(r"Substituting $%s$ and $%s$ results in:", r_mod, phi_mod)

J = jacobian([vx.subs({x: x2, y: y2}), vy.subs({x: x2, y: y2})], [r_strip, phi_strip])
s = ""
for i, row in enumerate(J.tolist()):
    for j, e in enumerate(row):
        s += (r"$J_{%d%d} = %s$"+"\n\n") %(i, j, sp.latex(e))

display(Markdown(s))
J = J.applyfunc(lambda c: c.doit())
J = sp.simplify(J)
latex(r"""
Carrying out the derivative gives the final jacobian converting from **STRIP-PC** to **MODULE-PC**:
$$J = %s$$
""", J)

Simplify a bit to ease calculation:

In [ ]:
A, B, C = sp.symbols("A B C")
A_expr = sp.simplify(sp.expand((Ox + r_strip*sp.cos(phi_strip - dphi))**2 
                               + (Oy + r_strip*sp.sin(phi_strip - dphi))**2))
B_expr = sp.cos(phi_strip - dphi)
C_expr = sp.sin(phi_strip - dphi)
Jsim = sp.simplify(sp.expand(J)).subs({A_expr: A, B_expr: B, C_expr: C})

latex("""$$J = %s$$""", Jsim)
latex(r"where: $A = %s \\ B = %s \\ C = %s$", A_expr, B_expr, C_expr)

In [ ]:
# generate some c++ code to avoid errors by re-typing:
s = ""
extocpp = lambda e: sp.printing.cxxcode(e.subs({"r_{strip}": "r_strip", "\Delta\phi": "dphi", 
                                                "\phi_{strip}": "phi_strip"}))
s += str("// " & "A = " + LineArray(pretty(A_expr.subs({r_strip: "rStrip", phi_strip: "phiStrip", dphi: "dPhi"}))))
s += "\n"
s += "double A = " + extocpp(A_expr) + ";"
s += "\n"*2
s += str("// " & "B = " + LineArray(pretty(B_expr.subs({r_strip: "rStrip", phi_strip: "phiStrip", dphi: "dPhi"}))))
s += "\n"
s += "double B = " + extocpp(B_expr) + ";"
s += "\n"*2
s += str("// " & "C = " + LineArray(pretty(C_expr.subs({r_strip: "rStrip", phi_strip: "phiStrip", dphi: "dPhi"}))))
s += "\n"
s += "double C = " + extocpp(C_expr) + ";"
s += "\n"*2
s += str("// " & "J = " + LineArray(pretty((Jsim.subs({r_strip: "rStrip", phi_strip: "phiStrip", dphi: "dPhi"})))))
s += "\n"
s += "Eigen::Matrix<double, 2, 2> J;\n"
for i, row in enumerate(Jsim.tolist()):
    for j, e in enumerate(row):
        s += "J(%d, %d) = "%(i,j) + extocpp(e)
        s += ";\n"
print_cpp(s)

# Intersection between line and shifted circle

For the parametrization in `AnnulusBoundsPC`, we need to figure out the edge points so we can do the decomposition into outward sectors. The points need to be in the **STRIP** system. To find the, we need to intersect a line (left and right angular edge of the module) and a shifted circle (inner and outer radial bounds, in the **MODULE**).

In [ ]:
x, y, m, b = sp.symbols("x y m b")
r, O_x, O_y = sp.symbols("r O_x O_y")
line_eq = m*x + b
#display(line_eq)
circle_eq = (x-O_x)**2 + (y-O_y)**2 - r**2
#display(circle_eq)

c_subs = circle_eq.subs(y, line_eq)
#display(c_subs)
sols_x = [sp.simplify(sp.expand(s.subs({b: 0}))) for s in sp.solve(c_subs, x)]
display(sols_x)

for s in [e.subs({b: 0}) for e in sols_x]: 
    print_cpp(sp.printing.cxxcode(s))
    print_cpp("// " & LineArray(pretty(s)))


Test that the above calculation actually works.

In [ ]:
mid = Point(0, -0.5)
radius = 1
circ = mid.buffer(radius)
out = Point(2*cos(pi/8), 2*sin(pi/8))
line = LineString([(0, 0), out])
subs = {O_x: mid.x, O_y: mid.y, b: 0, m: out.y/out.x, r: radius}
display(subs)

ix = line.intersection(circ.exterior)

fig = plt.figure()

ax = fig.add_subplot(111, aspect=1, xlim=(-5, 5), ylim=(-5, 5))
ax.add_patch(PolygonPatch(circ, fc="lightgrey"))

ax.plot(*line.xy, color="red")
for p in (ix if type(ix) is list else [ix]):
    #print("Point:", p.x, p.y)
    ax.plot(*p.xy, "o", color="orange", markersize=6)

save(fig, "line_circle_intersect.pdf")

vals_x = [x.subs(subs) for x in sols_x]
vals_y = [sp.solve(line_eq-y, y)[0].subs(subs).subs(x, vx) for vx in vals_x]

pts = [sp.Matrix([x, y]) for x, y in zip(vals_x, vals_y)]
outm = sp.Matrix([out.x, out.y])


the_point, = [p for p in pts if p.dot(sp.Matrix([1, 0])) > 0]
display(the_point)

assert abs(1-ix.x / the_point[0]) < 1e-2
assert abs(1-ix.y / the_point[1]) < 1e-2


# Production of test data for AnnulusDebugAlg

To validate the geometry implementation in C++, a reference is established using the `shapely` library. This library allows construction of the needed annulus-like shape using boolean operations, and then allows calculating the distance.

The first step is to produce the boolean shape using the same input parameters that we use for the C++ implementation. In fact, the code below produces C++ code with the input parameters specified here.

Using the boolean-constructed shape, we can then create a regular grid of test points, calculate their distances to the shape in python, and write out the results as test data for a C++ based test.

In [ ]:
def make_annulus(minR, maxR, phiMin, phiMax, phiAvg = 0, origin = (0, 0), plot=False, plot_margin=0.2, name="ab"):
    #print("phiMin", phiMin, "phiMax", phiMax)
    ox, oy = origin
    oR = sqrt(ox**2 + oy**2)
    ophi = atan2(oy, ox)
    porigin = Point(oR*cos(ophi - phiAvg), oR*sin(ophi - phiAvg))
    
    cin = porigin.buffer(minR)
    cout = porigin.buffer(maxR) 
    
    p1 = (maxR*2*cos(phiMax - phiAvg), maxR*2*sin(phiMax - phiAvg))
    p2 = (maxR*2*cos(phiMin - phiAvg), maxR*2*sin(phiMin - phiAvg))
    
    O = (0, 0)
    poly = Polygon([O, p1, p2])
    p_out = poly.boundary.intersection(cout.boundary)
    p_in = poly.boundary.intersection(cin.boundary)
    
    ring = cout.difference(cin)
    annulus = poly.intersection(ring)
    
    if plot:
        fig = plt.figure(figsize=(16, 7))
        def sp(lay, bounds, aspect=1):
            minx, miny, maxx, maxy = bounds
            ax = fig.add_subplot(lay, 
                             xlim=(minx-plot_margin, maxx+plot_margin), 
                             ylim=(miny-plot_margin, maxy+plot_margin), 
                             aspect=aspect)
            return ax
        
        
        ax = sp(131, cout.bounds)
        ax.add_patch(PolygonPatch(cout, fc="green"))
        ax.add_patch(PolygonPatch(cin, fc="blue"))
        ax.plot(*porigin.xy, "o", color="red")
        ax.plot(*O, "o", color="red", markerfacecolor='None')
        
        ax = sp(132, cout.union(poly).bounds)
        ax.add_patch(PolygonPatch(ring, fc="lightgrey"))
        
        ax.plot(*porigin.xy, "o", color="red")
        ax.plot(*O, "o", color="red", markerfacecolor='None')
        ax.plot(*poly.exterior.xy)    
        ax.plot([p.x for p in p_out], [p.y for p in p_out], "o", color="orange")
        ax.plot([p.x for p in p_in], [p.y for p in p_in], "o", color="orange")

        ax = sp(133, cin.union(annulus).bounds)
        ax.plot(*cin.boundary.xy, "--", color="grey")
        ax.plot(*cout.boundary.xy, "--", color="grey")
        ax.add_patch(PolygonPatch(annulus))
        ax.plot(*porigin.xy, "o", color="red")
        ax.plot(*O, "o", color="red", markerfacecolor='None')
        ax.plot(*LineString([O, p_in[0]]).xy, "b--")
        ax.plot(*LineString([O, p_in[1]]).xy, "b--")
        
        save(fig, "annulus_construction.pdf")

    cpp = """
Trk::AnnulusBoundsPC %s(%f, // minR
                        %f, // maxR
                        %f * M_PI, // phiMin
                        %f * M_PI, // phiMax
                        {%f, %f}, // origin
                        %f * M_PI /* phiAvg */);
"""
    cpp = cpp.strip() % (name, minR, maxR, phiMin/pi, phiMax/pi, ox, oy, phiAvg/pi)
    
    return annulus, cpp

# this constructs the annulus-like shape, with (arbitrary) parameters
annulus, annuluscpp = make_annulus(1, 2, -pi/8, +pi/8, origin = (0, -0.5), phiAvg=0, plot=True, name="asymT1Ab")
print_cpp(annuluscpp)

## Distance calculation

The distance that is calculated depends on the coordinate system. The local position argument of the `minDistance` method is expected to be in **STRIP** PC, but it probably makes the most sense to return cartesian distance anyway. In cartesian distance we can decompose the shape and find the closest point on each side.

In [ ]:
# this method generates a list of points around the bounding box of the geometry object with a 
# speficied margin, and calculates the closest points. A list of tuples 
def calc_distances(obj_orig, npoints, margin=0.2, trf=lambda x, y: (x, y), itrf=lambda x, y: (x, y)):
    result = []
    obj = transform(trf, obj_orig)
    gminx, gminy, gmaxx, gmaxy = obj_orig.bounds
    for gx in np.linspace(gminx-margin, gmaxx+margin, npoints):
        for gy in np.linspace(gminy-margin, gmaxy+margin, npoints):
            pointpc = Point(*trf(gx, gy))
            pointxy = Point(gx, gy)

            d = obj.boundary.project(pointpc)
            p = obj.boundary.interpolate(d)
            closest_point_coords = list(p.coords)[0]
            cls = Point(*itrf(*closest_point_coords))
            result.append((pointxy, cls))
    return result

In [ ]:
# number of test points to produce in each coordinate
npoints = 20
cart_points = calc_distances(annulus, npoints)
def polartrf(x, y):
    return tuple(Vec2(x, y).to_polar())
def carttrf(l0, l1):
    return tuple(Vec2(l0, l1).to_cartesian())
points_polar = calc_distances(annulus, npoints, trf=polartrf, itrf=carttrf)
annulus_polar = transform(polartrf, annulus)

In [ ]:
prog = tqdm(total=len(cart_points)*3, leave=False)

fig = plt.figure(figsize=(14, 7))

display(Markdown(r"""
The plots below illustrate the difference in distance calculation. The left side show cartesian closest points 
(and their distance), while the right side shows the same for polar calculation.
"""))

# cartesian
ax = fig.add_subplot(121, aspect=1, xlabel="x", ylabel="y", title="projection in XY")
points = calc_distances(annulus, npoints)
for p, cls in points:
    within = p.within(annulus)
    ax.plot(*p.xy, "o", color="green" if within else "red")
    ax.plot(*cls.xy, "o", color="blue")
    l = LineString([p, cls])
    ax.plot(*l.xy, linewidth=3, color="orange")
    prog.update()
ax.add_patch(PolygonPatch(annulus, fc="lightgrey"))

# polar
ax = fig.add_subplot(122, aspect=1, xlabel="x", ylabel="y", title="projection in PC")

for p, cls in points_polar: # points are not in polar, distances are
    within = p.within(annulus)
    ax.plot(*p.xy, "o", color="green" if within else "red")
    ax.plot(*cls.xy, "o", color="blue")
    l = LineString([p, cls])
    ax.plot(*l.xy, linewidth=3, color="orange")
    prog.update()
ax.add_patch(PolygonPatch(annulus, fc="lightgrey"))

save(fig, "distances_cart_pc.pdf")

display(Markdown(r"""
Note that the resulting connecting
lines are drawn in the cartesian frame for both cases, so they appear as straight lines in both cases. This
does not actually represent the distance being calculated. The plot below draws the connecting lines in a
polar projection, where they appear curved.
"""))

fig = plt.figure(figsize=(8, 8))

ax = fig.add_subplot(111, 
                 aspect=1, 
                 xlabel="r", ylabel="\phi",
                 rlim=(0, 3),
                 title="projection in PC",
                 projection="polar")
ax.set_thetamin(-70)
ax.set_thetamax(70)
annulus_pltpc = transform(lambda x, y: tuple(reversed(polartrf(x, y))), annulus)
ax.add_patch(PolygonPatch(annulus_pltpc, fc="lightgrey"))
for p, cls in points_polar: # points are not in polar, distances are
    ppc = Vec2(*reversed(polartrf(p.x, p.y)))
    clspc = Vec2(*reversed(polartrf(cls.x, cls.y)))
    within = p.within(annulus)
    ax.plot(*ppc.xy, "o", color="green" if within else "red")
    ax.plot(*clspc.xy, "o", color="blue")
    l = Line2(ppc, clspc)
    ax.plot(*l.xy, linewidth=3, color="orange")
    prog.update()
    
prog.close()
save(fig, "distance_polar_proj.pdf")

The implemented `minDistance` method is implemented consistently with the cartesian calculation shown above.

### Produce C++ output to create the tests more easily

This generates C++ code to reproduce the annulus bounds class, the test points, their respective closest points, the correct distance between the two, as well as whether the point is actually inside the bounds or not. **Very long output**.

In [ ]:
name = "asymT1"

cppstr = ""

cppstr += annuluscpp + "\n"*2

cppstr += make_cpp_vector("%sTestPoints"%name, 
                         "Amg::Vector2D", 
                         ["{%s}"%", ".join(map(str, (p.x, p.y))) for p, _ in points])
cppstr += "\n"*2
cppstr += make_cpp_vector("%sClosestPoints"%name, 
                         "Amg::Vector2D", 
                         ["{%s}"%", ".join(map(str, (p.x, p.y))) for _, p in points])
cppstr += "\n"*2
cppstr += make_cpp_vector("%sActDistances"%name, 
                         "double", 
                         [str(LineString([p, cls]).length) for p, cls in points])
cppstr += "\n"*2
cppstr += make_cpp_vector("%sActInsides"%name, 
                         "bool", 
                         ["true" if p.within(annulus) else "false" for p, _ in points], 
                         80)

print_cpp(cppstr)

# Coordinate transform in polar coordinates

If we want to do quick inside checks, parametrizing $r$ in **MODULE** as a function of $r$ and $\phi$ in **STRIP** system should save some time, since we don't have to do the full blown coordinate transform.

We take $x$, $y$ as the local coordinates in the **STRIP** system. The **MODULE** system is shifted by $S_x$, $S_y$ to give the coordinates $x'$, $y'$ in the **MODULE** system. 

$$
\left(
\begin{matrix}
x \\ y
\end{matrix}
\right)
+
\left(
\begin{matrix}
S_x \\ S_y
\end{matrix}
\right)
=
\left(
\begin{matrix}
x' \\ y'
\end{matrix}
\right)
$$

We are interested in the quantity $r'$, which is the radius of the point in question, with respect to the origin of the **MODULE** system.

\begin{align}
r' = \sqrt{x'^2 + y'^2} &= \sqrt{x^2 + S_x^2 + 2x S_x + y^2 + S_y^2 + 2y S_y} \\
&= \sqrt{ r_S^2 + r^2 + 2 r \cdot r_S (\cos\phi \cos\phi_S + \sin\phi\sin\phi_S) } \\
&= \sqrt{ r_S^2 + r^2 + 2r\cdot r_S \cos(\phi - \phi_S) }
\end{align}

using

\begin{equation}
2xS_x = 2rr_S\cos\phi\cos\phi_S\\
2yS_y = 2rr_S\sin\phi\sin\phi_S\\
\cos\phi \cos\phi \mp \sin\phi\sin\phi_S = \cos(\phi \pm \phi_S) \\
r_S^2 = S_x^2 + S_y^2 \\
r = x^2 + y^2
\end{equation}

The shifted angle $\phi'$ can be calculated as well in principle, but the sign convention seems to be different than what we're used to, and we don't need it for the check anyway. Therefore, only the calculation for $r'$ is implemented in C++.

Let's verify that this actually works:

In [ ]:
# given a point p1_c (in STRIP system) and shift in cartesian coordinates do the following:
def pol(p1_c, shift):
    # convert p1_c to polar STRIP
    #p1_pc = p1_c.to_polar()

    # calculate the polar coordinates of the shift (STRIP to MODULE)
    #shift_pc = shift.to_polar()
    
    # apply cartesian shift to cartesian p1, this gives p2 which is in MODULE
    p2_c = p1_c + shift
    # convert p2 to polar, in MODULE
    p2_pc = p2_c.to_polar()

    # calculate r' from the p1_c and shift, all in STRIP
    # the .phi and .r accessors give polar coordinates, but till in STRIP
    sqrt_arg = shift.r**2 + p1_c.r**2 + 2*shift.r*p1_c.r*cos(p1_c.phi - shift.phi)
    if abs(sqrt_arg) < 1e-10: sqrt_arg = 0
    try:
        # we can omit the sqrt for comparisons in C++.
        rnew = sqrt(sqrt_arg)
    except:
        print(shift.r**2, p1_c.r**2, 2*shift.r*p1_c.r*cos(p1_c.phi - shift.phi), sqrt_arg)
        raise

    # return the radius calculated by applying the shift in cartesian, and then converting
    # and also the radius as calculated directly from the r,phi STRIP coordinates
    return p2_c.r, rnew
    
for sx in range(-5, 5, 100):
    for sy in range(-5, 5, 100):
        for px in range(-5, 5, 100):
            for py in range(-5, 5, 100):
                ref, act = pol(Vec2(px, py), Vec2(sx, sy))
                assert ref - act < 1e-6, (ref, act)
print("ALL GOOD")

# Visual testing

The `AnnulusTestAlg` produces a few csv outputs. These files can be read in and plotted here. This gives the ability to visually verify that results are reasonable. The C++ algorithm throw random points around the bounds, tests whether the bounds consider the points to be inside according to several rules, and writes out the point along with that information.

In [ ]:
def read_csv(_file, *args, **kwargs):   
    file = os.path.join(run_dir, _file)
    print(file, "modified:", datetime.fromtimestamp(os.path.getmtime(file)))
    if not "sep" in kwargs: kwargs["sep"] = ";"
    return pd.read_csv(file, *args, **kwargs)

This displays the results of the methods `insideLoc1`, `insideLoc2` and `inside`, which return whether the coordinate given as an argument is inside the bounds in that direction, or both. The check is carried out against the polygon-based shape produced above, so we can plot the reference shape on top of the points that we've tested.

The methods have multiple modes:


- The tolerance modes take absolute tolerances. If a point is closer to the edge of the bounds that the tolerance, it   considered *inside*, else it is considered *outside*. The correlation of these tolerances is neglected.
  ```cpp
  bool Trk::AnnulusBoundsPC::inside(const Amg::Vector2D& locpo, double tol1, double tol2) const;
  ```

- `BoundaryCheck` mode accepts an object which describes how the boundary check is performed. This object can contain
  either absolute tolerances, in which case it performs identically as the mode above, or a covariance matrix.
  ```cpp
  bool Trk::AnnulusBoundsPC::inside(const Amg::Vector2D& locpo, const BoundaryCheck& bchk) const;
  ```
  In the covariance case, the Mahalanobis distance is calculated. The covariance is assumed to be given in **STRIP**
  polar coordinates. The Mahalanobis projection and distance calculation is performed on the angular edges using this
  covariance. The covariance matrix is then converted to **MODULE** polar coordinates, and the same projection and 
  distance calculation is performed. If the smallest Mahalanobis distance is smaller than the number of $\sigma$ 
  given in the `BoundaryCheck` object, it is considered inside.

In [ ]:
dftol12 = read_csv("vis_tol12.csv")

In [ ]:
fig = plt.figure(figsize=(16, 10))

insideAbs = dftol12.loc[dftol12['insideAbs'] == 1]
insideTolExcl = dftol12.loc[(dftol12['insideAbs'] != 1) & (dftol12["insideTol"] == 1)]
insideLoc1Abs = dftol12.loc[(dftol12["insideLoc1Abs"] == 1)]
insideLoc1Tol = dftol12.loc[(dftol12["insideLoc1Tol"] == 1) & (dftol12["insideLoc1Abs"] == 0)]
insideLoc2Abs = dftol12.loc[(dftol12["insideLoc2Abs"] == 1)]
insideLoc2Tol = dftol12.loc[(dftol12["insideLoc2Tol"] == 1) & (dftol12["insideLoc2Abs"] == 0)]

# loc 12
ax = fig.add_subplot(231, aspect=1, )
ax.add_patch(PolygonPatch(annulus, alpha=1, fill=False, linewidth=4, ec="green"))
ax.scatter(insideAbs["x"], insideAbs["y"], color="blue")
ax.scatter(insideTolExcl["x"], insideTolExcl["y"], color="red")

# loc 1
ax = fig.add_subplot(232, aspect=1)
ax.set_title("insideLoc1")
ax.add_patch(PolygonPatch(annulus, alpha=1, fill=False, linewidth=4, ec="green"))
ax.scatter(insideLoc1Abs["x"], insideLoc1Abs["y"], color="blue")
ax.scatter(insideLoc1Tol["x"], insideLoc1Tol["y"], color="red")
ax = fig.add_subplot(235, aspect=1, **boundlim(annulus))
ax.set_title("insideLoc1")
ax.add_patch(PolygonPatch(annulus, alpha=1, fill=False, linewidth=4, ec="green"))
ax.scatter(insideLoc1Abs["x"], insideLoc1Abs["y"], color="blue")
ax.scatter(insideLoc1Tol["x"], insideLoc1Tol["y"], color="red")

# loc 2
ax = fig.add_subplot(233, aspect=1)
ax.set_title("insideLoc2")
ax.add_patch(PolygonPatch(annulus, alpha=1, fill=False, linewidth=4, ec="green"))
ax.scatter(insideLoc2Abs["x"], insideLoc2Abs["y"], color="blue")
ax.scatter(insideLoc2Tol["x"], insideLoc2Tol["y"], color="red")
ax = fig.add_subplot(236, aspect=1, **boundlim(annulus))
ax.add_patch(PolygonPatch(annulus, alpha=1, fill=False, linewidth=4, ec="green"))
ax.set_title("insideLoc2")
ax.scatter(insideLoc2Abs["x"], insideLoc2Abs["y"], color="blue")
ax.scatter(insideLoc2Tol["x"], insideLoc2Tol["y"], color="red")

save(fig, "inside_tol.png")

The above plots show the results of the tolerance based checks. The green line is the reference shape, as rendered directly in python. The red points are considered *inside* the tolerance of that direction, the blue points are considered *inside* by absolute cuts (tolerance = 0) by the C++ implementation. 

The top left plot shows the result of `inside`. The other plots show `insideLoc1` and `insideLoc2`, which correspond to $r$ and $\phi$ in the **STRIP** system. The lower row zooms in on the plots above them. 

This tests the inside method in the covariance mode. An arbitrary covariance matrix is chosen.

In [ ]:
dfcov = read_csv("vis_cov.csv")

In [ ]:
colors = {0: "grey", 1: "yellow", 2: "green", 3: "pink", 4: "orange"}
dfcov["color"] = dfcov["side"].apply(lambda r: colors[r])

In [ ]:
insideAbs = dfcov[dfcov["inside"] == 1]
insideCov = dfcov[(dfcov["insideCov"] == 1) & (dfcov["inside"] == 0)]

fig = plt.figure(figsize=(10, 10))
ax = plt.subplot(111, aspect=1)
ax.scatter(insideAbs["x"], insideAbs["y"], color="blue")
insideCov.plot.scatter(x="x", y="y", c=insideCov["color"], ax=ax)
ax.add_patch(PolygonPatch(annulus, alpha=1, fill=False, linewidth=4, ec="green"))

save(fig, "inside_cov.png")

The blue points are considered absolute inside by the C++ implementation, while the grey points are considered inside based on the covariance check. The green line is the reference calculated and drawn in python.

The covariance based check takes into account both dimensions at the same time, explaining the *rounded* corners. This visualization appears to be reasonable

## Comparison to current implementation

Another comparison is the one to the current implementation. The implementation is such that an existing instance of `AnnulusBounds` can be used to construct an equivalent `AnnulusBoundsPC`. These two instances should then be geometrically identical. 

This test then checks whether the translation from the previous parametrization is correctly translated into the parameters required for the new parametrization.

This is tested by producing a cartesian instance and converting this instance to polar coordinates. Then random points are thrown in the **MODULE** cartesian frame and converted into the **STRIP** polar frame. The points are then tested against the respective implementation. Both results are written out, they should agree.

In [ ]:
dfcmp = read_csv("vis_cmp.csv")

In [ ]:
insideRef = dfcmp[dfcmp["insideRef"] == 1]
insidePC = dfcmp[dfcmp["inside"] == 1]

insideCorrect = dfcmp[(dfcmp["insideRef"] == dfcmp["inside"]) & (dfcmp["insideRef"] == 1)]
insideIncorrect = dfcmp[dfcmp["insideRef"] != dfcmp["inside"]]

fig = plt.figure(figsize=(13, 7))
ax = fig.add_subplot(121, aspect=1)
ax.scatter(insideRef["x"], insideRef["y"], color="orange", label="inside ref")
ax.scatter(insidePC["x"], insidePC["y"], color="blue", label="inside pc")

ys = np.linspace(-1, 11)
k_L = -6.6658
k_R = 2.73951
d_L = -1.31112
d_R = 0.556981
ax.plot((ys-d_L)/k_L, ys, c="grey")
ax.plot((ys-d_R)/k_R, ys, c="grey")

O_x = (d_L - d_R) / (k_R - k_L)
O_y = O_x*k_L + d_L

ms = 5
ax.plot(*Point(0, 0).buffer(5).exterior.xy, c="grey")
ax.plot(*Point(0, 0).buffer(10).exterior.xy, c="grey")
ax.plot(0, 0, "x", c="green", label="origin STRIP", markersize=ms)
ax.plot(O_x, O_y, "o", c="red", label="origin MODULE", markersize=ms)

ax.set_xlim(-5, 5)
ax.set_ylim(-1, 11)
ax.set_xlabel("x")
ax.set_ylabel("y")

ax.legend()

ax = fig.add_subplot(122, aspect=1)
ax.scatter(insideCorrect["x"], insideCorrect["y"], color="green", label="correct points")
ax.scatter(insideIncorrect["x"], insideIncorrect["y"], color="red", label="incorrect points")
ax.plot((ys-d_L)/k_L, ys, c="grey")
ax.plot((ys-d_R)/k_R, ys, c="grey")
ax.plot(*Point(0, 0).buffer(5).exterior.xy, c="grey")
ax.plot(*Point(0, 0).buffer(10).exterior.xy, c="grey")
ax.plot(0, 0, "x")
ax.plot(O_x, O_y, "o")
ax.set_xlim(-2, 4)
ax.set_ylim(4, 11)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.legend()

fig.tight_layout()
save(fig, "consistency_bounds.png")

The plots above show the thrown points, color coded depending on the status. The lines are drawn in python from (manually) extracted parameters. They indicate the nominal position of the bounds.

The left plot shows the origin of the two coordinate systems. The points considered inside by the reference implementation are shown in orange, the once considered inside by the PC implementation in blue. Only blue points are visible because the PC points are drawn on top.

The right plot shows points where both implementations agree in green, while points where they disagree are shown in red. (There are no red points visible, yes, they are drawn on top of the green points)

## Surface class is queried using isOnSurface

This part tests the translation of an instance of `AnnulusBounds` on a `PlaneSurface` into an `AnnulusBoundsPC` on a `DiscSurface`. To this end, instances of the previous implementation are produced with *random* transforms, and bounds parameters. These instances are then passed into a `DiscSurface` constructor which takes the transform from the previous `DiscSurface` and translates the bounds. (The call needs to manually extract the transform and the bounds). These two instances should agree on the results of the method `Surface::isOnSurface` for all points.

The parametrization of the cartesian annulus bounds is oriented towards the $y$ axis, whereas the polar cooridnate implementation is oriented toward the $y$ axis. This is accounted for by rotating the local surface frame so these axes align.

Random test points are thrown in local coordinates of the `PlaneSurface`, converted to global coordinates, and then fed into the `::isOnSurface()` methods of both instances. The results are written out. (The output also contains information on the corner points, origin shifts and stereo angle, which are used to draw the dashed reference lines.)

Any discrepancy should be seen in these random test points.

The test is done in C++ for 100 random instances and 10000 test points each. 20 instance results are plotted below.

In [ ]:
dfcmpsrf = read_csv("vis_cmp_srf.csv")

In [ ]:
def do_nr(ax, df, nr):
    onSurfaceRef = df[df["onSurfaceRef"] == 1]
    onSurfaceAct = df[df["onSurfaceAct"] == 1]
    onSurfaceCorrect = df[(df["onSurfaceAct"] == df["onSurfaceRef"]) & df["onSurfaceRef"] == 1]
    onSurfaceIncorrect = df[df["onSurfaceRef"] != df["onSurfaceAct"]]

    f = 1
    onSurfaceRef = onSurfaceRef.iloc[::f, :]
    onSurfaceAct = onSurfaceAct.iloc[::f, :]
    onSurfaceCorrect = onSurfaceCorrect.iloc[::f, :]
    onSurfaceIncorrect = onSurfaceIncorrect.iloc[::f, :]

    ax.set_xlabel("$l_x$")
    ax.set_ylabel("$l_y$")
    
    ref_corners = list(map(float, df.iloc[0]["ref_corners"].split(",")))
    act_corners = list(map(float, df.iloc[0]["act_corners"].split(",")))
    phiS = df.iloc[0]["phiS"]
    O_x = df.iloc[0]["Ox"]
    O_y = df.iloc[0]["Oy"]
    
    def minmax(a, amin, amax):
        return min(a, amin), max(a, amax)
    
    xmin, xmax = 1e12, -1e12
    ymin, ymax = 1e12, -1e12, 
    
    xmin, xmax = minmax(O_x, xmin, xmax)
    ymin, ymax = minmax(O_y, ymin, ymax)
    
    ppoints = []
    for j in range(3):
        ppoints.append(sp.Point3D(df.iloc[j]["globx"], df.iloc[j]["globy"], df.iloc[j]["globz"]))
    plane = sp.Plane(*ppoints)
    
    def draw(corners, color, n=0, rot=0, offset=(0, 0)):
        nonlocal xmin, xmax, ymin, ymax
        points_xyz = list(zip(corners[0::3], corners[1::3], corners[2::3]))
        points_xy = [(x, y) for x, y, z in points_xyz]
        
        points_xy = points_xy[n:] + points_xy[:n]
        
        # rotate the corners into local system so we can compare
        for idx in range(len(points_xy)):
            p_orig = Point(points_xy[idx])
            p_moved = Point(p_orig.x + offset[0], p_orig.y + offset[1])
            p_rotated = affinity.rotate(p_moved, rot, origin=(0, 0))
            points_xy[idx] = (p_rotated.x, p_rotated.y)
            
        
        
        l = LineString(points_xy + points_xy[0:1])
        poly = Polygon(points_xy)
        
        left_line = sp.Line(points_xy[0], points_xy[1])
        right_line = sp.Line(points_xy[2], points_xy[3])
        
        ax.plot(*LineString(points_xy[:2]).xy, "o", c=color)
        ax.plot(*LineString(points_xy[2:]).xy, "o", c=color, markerfacecolor="none")
        
        points_xy = list(map(lambda xy: Vec2(*xy), points_xy))
        
        right_dir = (points_xy[0] - points_xy[1]).normalized()
        left_dir = (points_xy[3] - points_xy[2]).normalized()
        
        ix = intersection(left_line, right_line)[0].n()
        xmin, xmax = minmax(ix.x, xmin, xmax)
        ymin, ymax = minmax(ix.y, ymin, ymax)
        ix = ix.x, ix.y
        ax.plot(*ix, "x", c=color)
        
        dminl, dmaxl = -20, 20
        dminr, dmaxr = dminl, dmaxl
        ax.plot(*LineString([ix, points_xy[0]]).xy, "--", c=color)
        ax.plot(*LineString([ix, points_xy[3]]).xy, "--", c=color)
        
        for p in points_xy[0], points_xy[3]:
            xmin, xmax = minmax(p[0], xmin, xmax)
            ymin, ymax = minmax(p[1], ymin, ymax)
                
        ax.add_patch(PolygonPatch(poly, alpha=.1, fill=True, linewidth=0, fc=color))
        
        O = Vec2(0, 0)
        ax.plot(*O.xy, "o", c=color)
        c_in = Point(O.x, O.y).buffer((points_xy[1] - O).norm)
        c_out = Point(O.x, O.y).buffer((points_xy[0] - O).norm)
        ax.plot(*c_in.exterior.xy, "--", c=color)
        ax.plot(*c_out.exterior.xy, "--", c=color)

    draw(ref_corners, color="g")
    draw(act_corners, color="r", n=2, rot=90-phiS/pi*180, offset=(-O_x, -O_y))
    
    for idx, row in onSurfaceCorrect.iterrows():
        xmin, xmax = minmax(row["locx"], xmin, xmax)
        ymin, ymax = minmax(row["locy"], ymin, ymax)
    
    ax.scatter(onSurfaceCorrect["locx"], onSurfaceCorrect["locy"], color="green", label="correct")
    ax.scatter(onSurfaceIncorrect["locx"], onSurfaceIncorrect["locy"], color="red", label="incorrect")
    
    if onSurfaceIncorrect.size == 0:
        ax.text(0.02, 0.97, "full match!", fontsize=12, transform=ax.transAxes, color="green")
    else:
        ax.text(0.02, 0.97, "mismatch!", fontsize=12, transform=ax.transAxes, color="red")
    
    brect = Polygon([(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin)])
    xmin, ymin, xmax, ymax = brect.buffer(0.5).exterior.bounds
    
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)

nplots = 20
prog = tqdm(total=nplots, leave=False)
js = range(0, dfcmpsrf["nr"].max()+1, 1)[:nplots]
gx = 4
gy = ceil((len(js)+1) / gx)
figw = 17
figh = gy/gx * figw
fig = plt.figure(figsize=(figw, figh))
for idx, j in enumerate(js):
    ax = fig.add_subplot(gy, gx, idx+1, aspect=1)
    df = dfcmpsrf[dfcmpsrf["nr"] == j]
    do_nr(ax, df, j)
    prog.update()

prog.close()
plt.show()

The plots above show the test points for the instances in local coordinates. The shapes of the modules are irregular and random as expected. The green points are agreed upon by both implementations, the red ones are incorrect. There are no red points visible.

This indicates that the translation `AnnulusBounds@PlaneSurface` $\to$ `AnnulusBoundsPC@DiscSurface` is reliable.